In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision torchaudio
!pip install tqdm matplotlib scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import time
import os
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from PIL import Image

In [ ]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of the current GPU
    gpu_name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Current GPU: {gpu_name}")

    # Get GPU device properties
    device = torch.device('cuda')
    print(f"Device type: {device.type}")
else:
    print("CUDA is not available. PyTorch will run on the CPU.")

Number of GPUs available: 1
Current GPU: Tesla T4
Device type: cuda


In [ ]:
# Standard image size for CNNs
image_size = 224

# Custom dataset class to handle corrupt images
class CustomImageFolder(ImageFolder):
    def __getitem__(self, index):
        try:
            return super().__getitem__(index)
        except Exception as e:
            print(f"Skipping corrupted image: {self.imgs[index][0]} - {e}")
            return self.__getitem__((index + 1) % len(self.imgs))  # Get next valid image

# Optimized Transformations (SAME FOR ALL MODELS)
train_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.Lambda(lambda img: img.convert('RGB')),  # Ensures all images are RGB
    transforms.RandomHorizontalFlip(p=0.5),  # Flip 50% of the time
    transforms.RandomRotation(10),  # Rotate within ±10 degrees
    transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0)),  # Random cropping
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),  # Color variation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard normalization
])

val_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.Lambda(lambda img: img.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Paths
dataset_path = "/content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset"
validation_path = "/content/drive/MyDrive/Waste Classification using Transfer Learning/Data Validation"

# Hyperparameters (Optimized for Performance)
batch_size = 32  # Changed from 64 to 32
num_workers = 4  # Parallel data loading

# Load dataset with corrupt image handling
train_dataset = CustomImageFolder(root=dataset_path, transform=train_transforms)
val_dataset = CustomImageFolder(root=validation_path, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Print class names to verify
print("Classes:", train_dataset.classes)

# Function to check for corrupt images before training
def check_corrupt_images(dataset_path):
    print("Checking dataset for corrupt images...")
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                try:
                    with Image.open(file_path) as img:
                        img.verify()  # Verify image integrity
                except:
                    print(f"Corrupt or non-image file found: {file_path}")

# Run dataset check
check_corrupt_images(dataset_path)
check_corrupt_images(validation_path)

print("Dataset loaded successfully!")

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Classes: ['cardboard', 'e-waste', 'glass', 'medical', 'metal', 'paper', 'plastic']
Checking dataset for corrupt images...
Corrupt or non-image file found: /content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset/paper/paper 2273.jpg
Corrupt or non-image file found: /content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset/cardboard/cardboard 1075.jpg
Corrupt or non-image file found: /content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset/e-waste/e-waste 1719.jpg
Checking dataset for corrupt images...
Dataset loaded successfully!


In [ ]:
def train_model(model, model_name, criterion, optimizer, train_loader, val_loader, device, num_epochs=25, patience=5, scheduler=None):
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    best_val_acc = 0  # Track best validation accuracy
    best_val_loss = float('inf')  # Track best validation loss
    patience_counter = 0

    # Create directory for saving models
    save_dir = os.path.join("/content/drive/MyDrive/Waste Classification using Transfer Learning/Saved Models", model_name)
    os.makedirs(save_dir, exist_ok=True)

    print("Starting training...")

    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}/{num_epochs} started...")
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if batch_idx % 10 == 0:
                print(f"Batch {batch_idx}/{len(train_loader)}: Loss = {loss.item():.4f}")

        train_loss = running_loss / len(train_loader)
        train_acc = (correct / total) * 100

        print("Training phase completed. Starting validation...")

        # Validation Phase
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for batch_idx, (inputs, labels) in enumerate(val_loader):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_acc = (val_correct / val_total) * 100

        # Compute Precision, Recall, and F1-score
        precision = precision_score(all_labels, all_preds, average='macro') * 100
        recall = recall_score(all_labels, all_preds, average='macro') * 100
        f1 = f1_score(all_labels, all_preds, average='macro') * 100

        # Store metrics
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)

        # Print Metrics for Each Epoch
        print(f"Epoch [{epoch}/{num_epochs}] completed.")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        print(f"Precision: {precision:.2f}% | Recall: {recall:.2f}% | F1-score: {f1:.2f}%")

        # Save checkpoint every 5 epochs
        if epoch % 5 == 0:
            checkpoint_path = os.path.join(save_dir, f'checkpoint_epoch_{epoch}.pth')
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'val_acc': val_acc
            }
            torch.save(checkpoint, checkpoint_path)
            print(f"Checkpoint saved at {checkpoint_path}!")

        # Save best model (based on validation accuracy or loss)
        if val_loss < best_val_loss or val_acc > best_val_acc:
            best_val_loss = min(best_val_loss, val_loss)
            best_val_acc = max(best_val_acc, val_acc)
            patience_counter = 0  # Reset patience counter

            best_model_path = os.path.join(save_dir, 'Model_ResNet34.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'val_acc': val_acc
            }, best_model_path)

            print(f"New best model saved at {best_model_path}!")
        else:
            patience_counter += 1  # Increment counter if no improvement

        # Early Stopping Condition
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch} due to no improvement in validation loss or accuracy.")
            break

    print("Training complete!")
    return train_losses, val_losses, train_accs, val_accs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import (
    resnet50, ResNet50_Weights,
    resnet34, ResNet34_Weights,
    efficientnet_b0, EfficientNet_B0_Weights,
    efficientnet_b3, EfficientNet_B3_Weights,
    vgg16, VGG16_Weights,
    densenet121, DenseNet121_Weights,
    mobilenet_v2, MobileNet_V2_Weights,
    inception_v3, Inception_V3_Weights
)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Select the model
selected_model_name = "ResNet-34"  # Change this to train other models

# Define models
models_list = {
    "ResNet-50": resnet50(weights=ResNet50_Weights.IMAGENET1K_V1),
    "ResNet-34": resnet34(weights=ResNet34_Weights.IMAGENET1K_V1),
    "EfficientNet-B0": efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1),
    "EfficientNet-B3": efficientnet_b3(weights=EfficientNet_B3_Weights.IMAGENET1K_V1),
    "VGG16": vgg16(weights=VGG16_Weights.IMAGENET1K_V1),
    "DenseNet-121": densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1),
    "MobileNet-V2": mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1),
    "Inception-V3": inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1)
}

# Get the selected model
model = models_list[selected_model_name]
print(f"\nTraining {selected_model_name}...\n")

# Adjust the final layer for 7 classes based on the model type
if "resnet" in selected_model_name.lower():
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 7)
elif "efficientnet" in selected_model_name.lower():
    num_features = model.classifier[1].in_features if hasattr(model.classifier, "__getitem__") else model.classifier.in_features
    model.classifier = nn.Linear(num_features, 7)
elif "densenet" in selected_model_name.lower():
    num_features = model.classifier.in_features
    model.classifier = nn.Linear(num_features, 7)
elif "vgg" in selected_model_name.lower():
    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_features, 7)
elif "mobilenet" in selected_model_name.lower():
    num_features = model.classifier[1].in_features if hasattr(model.classifier, "__getitem__") else model.classifier.in_features
    model.classifier = nn.Linear(num_features, 7)
elif "inception" in selected_model_name.lower():
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 7)
    if model.aux_logits:  # Inception has an auxiliary classifier
        num_features_aux = model.AuxLogits.fc.in_features
        model.AuxLogits.fc = nn.Linear(num_features_aux, 7)

# Move model to the device (CPU/GPU)
model = model.to(device)
model.train()  # Ensure the model is in training mode

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Train the model
train_losses, val_losses, train_accs, val_accs = train_model(
    model=model,
    model_name=selected_model_name,
    criterion=criterion,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    num_epochs=25,
    patience=5,
    scheduler=scheduler
)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 121MB/s]
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 142MB/s]
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 137MB/s]
Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-b3899882.pth
100%|██████████| 47.2M/47.2M [00:00<00:00, 165MB/s]
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 168MB/s]
Downloading: "h


Training ResNet-34...

Starting training...

Epoch 1/25 started...


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Batch 0/455: Loss = 2.2139
Batch 10/455: Loss = 1.1867
Batch 20/455: Loss = 1.2141
Batch 30/455: Loss = 0.9273
Batch 40/455: Loss = 0.5634


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Batch 20/455: Loss = 0.0413
Batch 30/455: Loss = 0.1667
Batch 40/455: Loss = 0.0771
Batch 50/455: Loss = 0.0612
Batch 60/455: Loss = 0.3386
Batch 70/455: Loss = 0.0513
Batch 80/455: Loss = 0.1200
Batch 90/455: Loss = 0.2461
Batch 100/455: Loss = 0.0614
Batch 110/455: Loss = 0.0338
Batch 120/455: Loss = 0.2265


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Batch 130/455: Loss = 0.1169
Batch 140/455: Loss = 0.1255
Batch 150/455: Loss = 0.0247
Batch 160/455: Loss = 0.0602
Batch 170/455: Loss = 0.0545
Batch 180/455: Loss = 0.0215
Batch 190/455: Loss = 0.1042
Batch 200/455: Loss = 0.0536
Skipping corrupted image: /content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset/e-waste/e-waste 1719.jpg - cannot identify image file <_io.BufferedReader name='/content/drive/MyDrive/Waste Classification using Transfer Learning/Dataset/e-waste/e-waste 1719.jpg'>
Batch 210/455: Loss = 0.2342
Batch 220/455: Loss = 0.3033
Batch 230/455: Loss = 0.1057
Batch 240/455: Loss = 0.2040
Batch 250/455: Loss = 0.0619
Batch 260/455: Loss = 0.0611
Batch 270/455: Loss = 0.0677
Batch 280/455: Loss = 0.0644
Batch 290/455: Loss = 0.3886
Batch 300/455: Loss = 0.0290
Batch 310/455: Loss = 0.0796
Batch 320/455: Loss = 0.2359
Batch 330/455: Loss = 0.0519
Batch 340/455: Loss = 0.1004
Skipping corrupted image: /content/drive/MyDrive/Waste Classification using T

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [9/25] completed.
Train Loss: 0.1525 | Train Acc: 95.00%
Val Loss: 0.3832 | Val Acc: 89.22%
Precision: 89.34% | Recall: 88.61% | F1-score: 88.89%
Early stopping at epoch 9 due to no improvement in validation loss or accuracy.
Training complete!


In [ ]:
import torch
# Define the folder path
save_path = "/content/drive/MyDrive/Waste Classification using Transfer Learning/Saved Models"
# os.makedirs(save_path, exist_ok=True)  # Create folder if it doesn't exist
torch.save(model, os.path.join(save_path, "Model_ResNet-34.pth"))
print(f"Model saved successfully in {save_path}/Model_ResNet-34.pth")

Model saved successfully in /content/drive/MyDrive/Waste Classification using Transfer Learning/Saved Models/Model_ResNet-34.pth
